<a href="https://colab.research.google.com/github/Engineer-Ayesha-Shafique/Brain-Tumor-Segmentation-and-Detection-using-UNET-and-Watershed-in-Python/blob/main/Brain_Tumor_Segmentation_and_Detection_using_UNET_and_Watershed_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive' ,force_remount=True)

Mounted at /content/drive


#MOE




In [3]:
import os
BASE_MODEL_PATH = "/content/drive/MyDrive/Project_41725"
MODEL_1 = os.path.join(BASE_MODEL_PATH, "files" , "UNetModified" , "model.h5")
MODEL_2 = os.path.join(BASE_MODEL_PATH, "files" , "LinkNet" , "model.h5")
MODEL_3 = os.path.join(BASE_MODEL_PATH, "files", "aug_pass" , "model_512.h5")
MODEL_4 = os.path.join(BASE_MODEL_PATH, "files", "DG" , "model.h5")
MODEL_5 = os.path.join(BASE_MODEL_PATH, "files", "DG" , "model_256.h5")

In [4]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ TensorFlow will use {len(gpus)} GPU(s)")
else:
    print("⚠️ No GPU detected for TensorFlow")


✅ TensorFlow will use 1 GPU(s)


In [5]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBClassifier
from tqdm import tqdm
import imageio

In [6]:
# ── 1) Imports & constants ─────────────────────────────────────────────────────

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Conv2D, Activation, Lambda
)

INPUT_SHAPE = (224, 224, 3)
NUM_CLASSES = 5

In [7]:
!pip install --quiet segmentation-models tensorflow-addons


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [8]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation,
    MaxPool2D
)
from tensorflow.keras.applications import EfficientNetB4

sm.set_framework('tf.keras')

BASE_MODEL_PATH = "/content/drive/MyDrive/Project_41725"
MODEL_1_UNet = os.path.join(BASE_MODEL_PATH, "files" , "UNetModified" , "model.h5")
MODEL_2_LINKNET = os.path.join(BASE_MODEL_PATH, "files" ,"LinkNet", "modell.h5")
MODEL_3_AttentionUNet = os.path.join(BASE_MODEL_PATH, "files", "aug_pass", "model_512.h5")
MODEL_4_DGNet = MODEL_4
MODEL_5_Upp =  MODEL_5
INPUT_SHAPE = (512, 512, 3)
ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"


Segmentation Models: using `tf.keras` framework.


In [9]:
# In a notebook cell, before any cv2 imports:
!pip install --quiet opencv-python-headless
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1) Import TF first and set memory growth (fallback to experimental)
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception as e:
            print(f"[GPU] Could not set memory growth: {e}")
    print(f"[GPU] Detected {len(gpus)} GPU(s).")

# 2) Now import the rest
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger, ReduceLROnPlateau,
    EarlyStopping, TensorBoard
)
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBClassifier
from tqdm import tqdm
import imageio
""" Global parameters """
Height = 512
Width  = 512

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path_dataset, val_ratio=0.2, test_ratio=0.1, random_state=42):
    img_dir  = os.path.join(path_dataset, "images")
    mask_dir = os.path.join(path_dataset, "masks")

    image_exts = ('.png', '.jpg', '.jpeg')
    images = sorted([
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_exts)
    ])


    mask_exts = ('.tif', '.tiff')
    masks = sorted([
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if f.lower().endswith(mask_exts)
    ])

    if not images:
        raise RuntimeError(f"No images found in {img_dir}")
    if not masks:
        raise RuntimeError(f"No masks found in {mask_dir}")

    images_lookup = {
        os.path.splitext(os.path.basename(p))[0]: p
        for p in images
    }
#MildDemented_0a7b1321-eba0-40dc-85b8-de34241554a2_jpg.rf.70370fcd9ac0fb82f91d27f558053d84.tif
    pairs = []
    for m in masks:
        base = os.path.splitext(os.path.basename(m))[0]
        if base.endswith('_mask'):
            root = base[:-5]
            img_path = images_lookup.get(root)
            if img_path:
                pairs.append((img_path, m))

    if not pairs:
        raise RuntimeError("No matching image↔mask pairs found!")

    trainval, test = train_test_split(pairs, test_size=test_ratio, random_state=random_state)
    val_frac = val_ratio / (1.0 - test_ratio)
    train, val = train_test_split(trainval, test_size=val_frac, random_state=random_state)

    train_imgs, train_masks = zip(*train)
    val_imgs,   val_masks   = zip(*val)
    test_imgs,  test_masks  = zip(*test)

    return (
        list(train_imgs), list(train_masks),
        list(val_imgs),   list(val_masks),
        list(test_imgs),  list(test_masks)
    )

def read_image_file(path_img):
    path = path_img.decode('utf-8') if isinstance(path_img, bytes) else path_img
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height)) / 255.0
    return img.astype(np.float32)

def read_mask_file(path_mask):
    path = path_mask.decode('utf-8') if isinstance(path_mask, bytes) else path_mask
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[..., 0]
    mask = cv2.resize(mask, (Width, Height), interpolation=cv2.INTER_NEAREST)

    # ✅ Clip to [0, 4], not [0, 3]:
    mask = np.clip(mask, 0, NUM_CLASSES-1)

    # Convert to integer class IDs
    mask = mask.astype(np.int32)

    # Add the channel dim back
    return mask[..., None]

def tf_parse(path_img, path_mask):
    # Tell numpy_function: img→float32, mask→int32
    img, mask = tf.numpy_function(
        lambda pi, pm: (read_image_file(pi), read_mask_file(pm)),
        [path_img, path_mask],
        [tf.float32, tf.int32]          # <-- second output is now int32
    )
    img.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return img, mask

def tf_dataset(inputs, targets, batch_size=2):
    ds = tf.data.Dataset.from_tensor_slices((inputs, targets))
    ds = ds.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def edit_mask_images(mask_dir, func, output_dir=None):
    out_dir = output_dir or mask_dir
    create_folder(out_dir)
    exts = ('.tif')
    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(exts):
            continue
        in_path  = os.path.join(mask_dir, fname)
        mask     = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
        edited   = func(mask)
        cv2.imwrite(os.path.join(out_dir, fname), edited)
    print(f"[edit_mask_images] Applied edits to masks in {out_dir}")

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_folder("/content/drive/MyDrive/Project_41725/files")

    batch_size = 16
    lr         = 1e-4
    epochs_num     = 100
    path_model = os.path.join("/content/drive/MyDrive/Project_41725/files", "model.h5")
    path_csv   = os.path.join("/content/drive/MyDrive/Project_41725/files", "log.csv")

    ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(ds_path)

    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    train_ds = tf_dataset(train_imgs, train_masks, batch_size=batch_size)
    val_ds   = tf_dataset(val_imgs,   val_masks,   batch_size=batch_size)
    test_ds  = tf_dataset(test_imgs,  test_masks,  batch_size=batch_size)

    # Build, compile and fit your model below:
    # model = Unet_model((Height, Width, 3))
    # model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coeff])
    # callbacks = [ModelCheckpoint(...), ...]
    # model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)


[GPU] Detected 1 GPU(s).
Train: 1216, Val: 348, Test: 174


In [10]:
import pandas as pd

# assuming you’ve re-run the load_dataset cell in XGBoost_ensemble.ipynb
# so train_imgs, train_masks, test_imgs are available:

pd.DataFrame({'image_path': train_imgs}).to_csv('train_images.csv', index=False)
pd.DataFrame({'mask_path':  train_masks}).to_csv('train_masks.csv',  index=False)
pd.DataFrame({'image_path': test_imgs }).to_csv('test_images.csv',  index=False)
print("Manifest CSVs written.")

Manifest CSVs written.


In [11]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
)
import os
# Make sure segmentation_models picks up tf.keras, not plain keras
os.environ["SM_FRAMEWORK"] = "tf.keras"

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
import segmentation_models as sm

# Hyper‑params & paths
NUM_CLASSES = 5
INPUT_SHAPE = (512, 512, 3)
EPOCHS      = 160
path_model  = "/content/drive/MyDrive/Project_41725/XGBoost/files/model.h5"
path_csv    = "/content/drive/MyDrive/Project_41725/XGBoost/files/log.csv"

# 0) kill any existing graph/state
tf.keras.backend.clear_session()

# 3) Define losses & metrics
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=NUM_CLASSES)
    y_true_f = tf.reshape(y_true_o, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred,   [-1, NUM_CLASSES])
    inter = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    denom = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * inter + smooth) / (denom + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

class MeanIoUArgMax(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes=NUM_CLASSES, name="mean_iou", dtype=None):
        super().__init__(num_classes=num_classes, name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

for name, fn in sm.losses.__dict__.items():
    get_custom_objects()[name] = fn
for name, fn in sm.metrics.__dict__.items():
    get_custom_objects()[name] = fn
get_custom_objects().update({
    'dice_coef': dice_coef,
    'combined_loss': combined_loss,
    'MeanIoUArgMax': MeanIoUArgMax
})



In [12]:
!pip install --quiet mlxtend xgboost tqdm opencv-python-headless

In [13]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold
from xgboost import XGBClassifier

model1 = load_model(
    MODEL_1_UNet,
    compile=False
)
model2 = load_model(
    MODEL_2_LINKNET,
    compile=False
)
model3 = load_model(
    MODEL_3_AttentionUNet,
    compile=False
)
model4 = load_model(
    MODEL_4_DGNet,
    compile=False
)
model5 = load_model(
    MODEL_5_Upp,
    compile=False
)
model1 = tf.keras.models.load_model(MODEL_1_UNet, compile=False)
model2 = tf.keras.models.load_model(MODEL_2_LINKNET, compile=False)
model3 = tf.keras.models.load_model(MODEL_3_AttentionUNet, compile=False)
model4 = tf.keras.models.load_model(MODEL_4_DGNet, compile=False)
model5 = tf.keras.models.load_model(MODEL_5_Upp, compile=False)

In [14]:
!pip install opencv-python-headless


In [15]:
for idx, m in enumerate([model1, model2, model3,model4,model5], 1):
    print(f"Model{idx}.input_shape →", m.input_shape)

Model1.input_shape → (None, 224, 224, 3)
Model2.input_shape → (None, 512, 512, 3)
Model3.input_shape → (None, None, None, 3)
Model4.input_shape → (None, 224, 224, 3)
Model5.input_shape → (None, None, None, 3)


In [16]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.3 MB/s eta 0:00:00


In [17]:
import cv2
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt

# your five TF/Keras models
base_models = [model1, model2, model3, model4, model5]

# fallback for any dynamic‐shape models
_manual_map = {
    model1: (224, 224),
    model2: (512, 512),
    model3: (512, 512),
    model4: (224, 224),
    model5: (512, 512),
}

def get_native_size(m):
    h, w = m.input_shape[1], m.input_shape[2]
    if h is None or w is None or h == 0 or w == 0:
        return _manual_map[m]
    return (h, w)

def soft_voting_mask(img):
    H, W = img.shape[:2]
    C = base_models[0].output_shape[-1]
    acc = np.zeros((H, W, C), dtype=np.float32)
    for m in base_models:
        h_n, w_n = get_native_size(m)
        inp = cv2.resize(img, (w_n, h_n), interpolation=cv2.INTER_AREA)
        pm  = m.predict(inp[None], verbose=0)[0]
        for c in range(C):
            acc[..., c] += cv2.resize(pm[..., c], (W, H), interpolation=cv2.INTER_LINEAR)
    acc /= len(base_models)
    mask = np.argmax(acc, axis=-1).astype(np.uint8)
    conf = np.max(acc, axis=-1)
    return mask, conf

def ensemble_predict(image):
    img = np.array(image)
    img_f = img.astype(np.float32)/255.0
    mask, conf = soft_voting_mask(img_f)

    # red overlay
    overlay = img.copy()
    overlay[mask>0] = [255,0,0]
    mask_overlay = cv2.addWeighted(overlay, 0.5, img, 0.5, 0)

    # confidence heatmap
    conf_img = (conf*255).astype(np.uint8)
    conf_bgr = cv2.applyColorMap(conf_img, cv2.COLORMAP_HOT)
    conf_vis = conf_bgr[..., ::-1]

    # class percentages
    total = mask.size
    lines = [f"Class {cls}: {100*np.sum(mask==cls)/total:4.1f}%"
             for cls in range(mask.max()+1)]
    pct = "\n".join(lines)

    return mask_overlay, conf_vis, pct

css = """
.gradio-container {
  max-width: 900px !important;
  margin: auto;
  padding: 2rem;
  background: #ffffff;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.1);
}
#mask-out, #conf-out {
  border-radius: 8px;
  box-shadow: 0 2px 6px rgba(0,0,0,0.1);
}
.gr-button {
  background-color: #e63946;
  color: white;
  border: none;
  padding: 0.75rem 1.5rem;
  border-radius: 6px;
  font-size: 1rem;
}
.gr-button:hover {
  background-color: #d62828;
}
.gr-box {
  background: #f8f9fa;
  border-radius: 8px;
  padding: 1rem;
  box-shadow: inset 0 1px 3px rgba(0,0,0,0.05);
}
"""

with gr.Blocks(css=css, theme=gr.themes.Base()) as demo:
    gr.Markdown("<h1 style='text-align:center;color:#333;'>🧠 Soft-Voting Ensemble Demo</h1>")
    gr.Markdown("<p style='text-align:center;color:#555;'>Upload an image to get a red mask overlay, confidence heatmap, & class breakdown.</p>")

    with gr.Row():
        inp = gr.Image(type="pil", label="Input Image", elem_id="input-img")
    with gr.Row():
        with gr.Column():
            gr.Markdown("**Mask Overlay**", elem_id="mask-title")
            m_out = gr.Image(type="numpy", label=None, elem_id="mask-out")
        with gr.Column():
            gr.Markdown("**Confidence Heatmap**", elem_id="conf-title")
            c_out = gr.Image(type="numpy", label=None, elem_id="conf-out")
    pct = gr.Textbox(label="Class Distribution", interactive=False, elem_id="dist-box")

    inp.change(fn=ensemble_predict,
               inputs=inp,
               outputs=[m_out, c_out, pct],
               show_progress=True)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://431c0e81bc80ef05df.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
